# Prep for Transform

In [1]:
import pathlib
import sys
import pickle

from typing import List

import numpy as np
import pandas as pd

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# add the 'src' directory to path to import modules
src_dir = pathlib.Path().cwd().resolve().parent / 'src'
#src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)
# import my class code from the source
# %aimport src-dir.filename

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

PROJECT_DIR = pathlib.Path.cwd().resolve().parent
MERGED_DATA_DIR = PROJECT_DIR / 'data' / '04-merged'
PREPPED_DATA_DIR = PROJECT_DIR / 'data' / '05-prepped-for-transform'

In [2]:
nations = ['germany', 'united-kingdom', 'spain', 'italy',
          'france', 'united-kingdom', 'germany', 'netherlands',
          'russian-federation', 'scotland', 'portugal', 'switzerland',
          'belgium', 'turkey', 'poland', 'united-kingdom']

# poland, switzerland are multileague

leagues = ['bundesliga', 'english-premier-league', 'la-liga', 'serie-a',
           'ligue-1', 'english-championship', 'bundesliga-2', 'eredivisie',
           'premier-league', 'premiership', 'primeira-liga', 'super-league',
           'first-division-a', 'super-lig', 'ekstraklasa', 'one']

seasons = ['2000-2001', '2001-2002', '2002-2003', '2003-2004',
           '2004-2005', '2005-2006', '2006-2007', '2007-2008',
           '2008-2009', '2009-2010', '2010-2011', '2011-2012',
           '2012-2013', '2013-2014', '2014-2015', '2015-2016',
           '2016-2017', '2017-2018']

## Load and Review Features

+ Get fduk csvs - cll this merged_csv (still within season)
+ Get Image Paths
+ Write image refs to merged_csv
+ save new csv and copy images based on relative address in merged_csv
+ Get odds data
+ Merge into merged_csv
+ Write merged_csv

In [3]:
nations = ['united-kingdom']
leagues = ['english-premier-league']

In [4]:
def form_fdcu_fps(MERGED_DATA_DIR, nation, league, seasons):
    """
    Accepts directory names to enable reach into raw data directory
    Returns full filepaths of the data files
    This is different - different signature no source - to other versions - needs to be standardized
    """
    fps = []
    fdcu_specifics = ['football-data-co-uk', 'season-data']
    for season in seasons:
        fn = season + '.csv'
        stub = MERGED_DATA_DIR / nation / league / season
        fp = stub / fn
        #if fp.is_file():
        if fp.exists():
            fps.append(fp)
    return fps

#fps = form_fdcu_fps(MERGED_DATA_DIR, nations[0], leagues[0], seasons)

In [5]:
all_fps = []
for nation, league in zip(nations, leagues):
    fps = form_fdcu_fps(MERGED_DATA_DIR, nation, league, seasons)
    all_fps.extend(fps)

print(len(all_fps))

11


In [6]:
season_dfs = []
for fp in all_fps:
    season_df = pd.read_csv(fp, parse_dates=['date'], index_col=None)
    season_dfs.append(season_df)
season_dfs[0].head()

,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_yellowCards,a,odds_awin_bet365_ic,odds_draw_bet365_ic,odds_hwin_bet365_ic,odds_awin_BW,odds_draw_BW,odds_hwin_BW,n_Bb1X2,n_BbAsian,BbAsian_handicap,odds_ftgoalsu2.5_bbmean,odds_ftgoalso2.5_bbmean,odds_awin_bbmean,odds_asianaway_bbmean,odds_asianhome_bbmean,odds_draw_bbmean,odds_hwin_bbmean,odds_ftgoalsu2.5_bbmax,odds_ftgoalso2.5_bbmax,odds_awin_bbmax,odds_asianaway_bbmax,odds_asianhome_bbmax,odds_draw_bbmax,odds_hwin_bbmax,n_BbOU,date,a_ftGoals,h_ftGoals,ftResult,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,a_htGoals,h_htGoals,h_yellowCards,h,odds_awin_IW,odds_draw_IW,odds_hwin_IW,odds_awin_LB,odds_draw_LB,odds_hwin_LB,odds_awin_VC,odds_draw_VC,odds_hwin_VC,odds_awin_WH,odds_draw_WH,odds_hwin_WH,league_ic,nation_ic,season_ic,clodds_away_pinn,clodds_draw_pinn,clodds_hwin_pinn,shotmap_path,heatmap_path,id_fifa,nation_fdcu,league_fdcu,season_fdcu,odds_hwin_pinn,odds_draw_pinn,odds_awin_pinn,odds_hwin_bet365_fdcu,odds_draw_bet365_fdcu,odds_awin_bet365_fdcu
0,2.0,11.0,0.0,17.0,7.0,2.0,liverpool,1.90,3.25,4.00,1.95,3.20,3.65,43.0,26.0,0.50,1.63,2.14,1.95,1.95,1.85,3.23,3.67,1.70,2.30,2.10,2.07,1.95,3.40,4.00,39.0,2007-08-11,2.0,1.0,A,4.0,18.0,0.0,10.0,6.0,1.0,0.0,4.0,aston-villa,1.9,3.2,3.60,1.90,3.2,3.50,2.00,3.20,3.5,NaN,NaN,NaN,english-premier-league,united-kingdom,2007-2008,NaN,NaN,NaN,shotmaps/2007-2008__2007-08-11__aston-villa__l...,heatmaps/2007-2008__2007-08-11__aston-villa__l...,eng-pl,united-kingdom,english-premier-league,2007-2008,3.95,3.37,2.02,3.40,3.20,2.0
1,3.0,16.0,0.0,7.0,5.0,1.0,newcastle-united,2.75,3.20,2.50,2.70,3.20,2.40,45.0,26.0,0.00,1.65,2.10,2.80,2.04,1.75,3.18,2.41,1.75,2.33,3.05,2.18,1.83,3.25,2.60,40.0,2007-08-11,3.0,1.0,A,4.0,15.0,0.0,13.0,9.0,3.0,0.0,1.0,bolton-wanderers,3.0,3.1,2.20,2.75,3.0,2.37,2.90,3.25,2.3,NaN,NaN,NaN,english-premier-league,united-kingdom,2007-2008,NaN,NaN,NaN,shotmaps/2007-2008__2007-08-11__bolton-wandere...,heatmaps/2007-2008__2007-08-11__bolton-wandere...,eng-pl,united-kingdom,english-premier-league,2007-2008,2.60,3.10,2.90,2.29,3.20,2.8
2,6.0,17.0,0.0,12.0,6.0,2.0,portsmouth,2.40,3.25,2.80,2.25,3.15,2.95,43.0,25.0,0.00,1.63,2.13,2.38,1.71,2.09,3.19,2.85,1.90,2.35,2.55,1.76,2.26,3.30,3.25,39.0,2007-08-11,2.0,2.0,D,6.0,14.0,0.0,12.0,5.0,1.0,1.0,1.0,derby,2.4,3.1,2.70,2.37,3.2,2.60,2.40,3.25,2.7,NaN,NaN,NaN,english-premier-league,united-kingdom,2007-2008,NaN,NaN,NaN,shotmaps/2007-2008__2007-08-11__derby__portsmo...,heatmaps/2007-2008__2007-08-11__derby__portsmo...,eng-pl,united-kingdom,english-premier-league,2007-2008,3.20,3.20,2.35,2.50,3.25,2.5
3,2.0,13.0,0.0,14.0,4.0,0.0,wigan-athletic,5.50,3.40,1.66,5.00,3.40,1.65,44.0,26.0,-0.75,1.73,2.00,5.25,1.94,1.94,3.45,1.66,1.83,2.20,6.00,2.02,1.97,3.65,1.75,40.0,2007-08-11,1.0,2.0,H,6.0,8.0,0.0,12.0,8.0,0.0,1.0,0.0,everton,4.5,3.5,1.65,4.50,3.5,1.61,5.00,3.40,1.7,NaN,NaN,NaN,english-premier-league,united-kingdom,2007-2008,NaN,NaN,NaN,shotmaps/2007-2008__2007-08-11__everton__wigan...,heatmaps/2007-2008__2007-08-11__everton__wigan...,eng-pl,united-kingdom,english-premier-league,2007-2008,1.73,3.50,5.40,1.62,3.50,5.0
4,3.0,16.0,0.0,4.0,4.0,4.0,blackburn-rovers,2.87,3.25,2.37,2.80,3.15,2.35,43.0,26.0,0.00,1.67,2.08,2.81,1.97,1.77,3.18,2.41,1.75,2.30,2.95,2.10,1.90,3.25,2.70,39.0,2007-08-11,2.0,1.0,A,13.0,16.0,0.0,10.0,6.0,0.0,1.0,3.0,middlesbrough,2.8,3.1,2.30,2.75,3.2,2.25,2.65,3.25,2.5,NaN,NaN,NaN,english-premier-league,united-kingdom,2007-2008,NaN,NaN,NaN,shotmaps/2007-2008__2007-08-11__middlesbrough_...,heatmaps/2007-2008__2007-08-11__middlesbrough_...,eng-pl,united-kingdom,english-premier-league,2007-2008,2.57,3.20,2.95,2.25,3.25,2.8


In [7]:
season_dfs[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 74 columns):
a_corners                  380 non-null float64
a_fouls                    380 non-null float64
a_redCards                 380 non-null float64
a_shots                    380 non-null float64
a_shotsOnTarget            380 non-null float64
a_yellowCards              380 non-null float64
a                          380 non-null object
odds_awin_bet365_ic        380 non-null float64
odds_draw_bet365_ic        380 non-null float64
odds_hwin_bet365_ic        380 non-null float64
odds_awin_BW               380 non-null float64
odds_draw_BW               380 non-null float64
odds_hwin_BW               380 non-null float64
n_Bb1X2                    380 non-null float64
n_BbAsian                  380 non-null float64
BbAsian_handicap           380 non-null float64
odds_ftgoalsu2.5_bbmean    380 non-null float64
odds_ftgoalso2.5_bbmean    380 non-null float64
odds_awin_bbmean           380 n

### Select Columns, Tidy up Column names

In [8]:
new_dfs = []
for season_df_orig in season_dfs:
    season_df = season_df_orig.copy(deep=True)
    keepers = ['nation_ic', 'league_ic', 'season_ic', 'date', 'h', 'a',
              'h_ftGoals', 'a_ftGoals',
              'h_corners', 'h_fouls', 'h_redCards', 'h_shots', 'h_shotsOnTarget', 'h_htGoals', 'h_yellowCards',
              'a_corners', 'a_fouls', 'a_redCards', 'a_shots', 'a_shotsOnTarget', 'a_htGoals', 'a_yellowCards',
              'odds_hwin_bbmean', 'odds_draw_bbmean', 'odds_awin_bbmean',
              'odds_hwin_bet365_ic', 'odds_draw_bet365_ic','odds_awin_bet365_ic',
              'odds_hwin_pinn', 'odds_draw_pinn', 'odds_awin_pinn',
              'odds_hwin_bbmax', 'odds_draw_bbmax', 'odds_awin_bbmax']
    
    rename_d = {'nation_ic': 'nation', 'league_ic': 'league', 'season_ic': 'season',
                'odds_hwin_bbmean': 'oddsHwinMean', 'odds_draw_bbmean': 'oddsDrawMean',
                'odds_awin_bbmean': 'oddsAwinMean',
               'odds_hwin_bet365_ic': 'oddsHwinBet', 'odds_draw_bet365_ic': 'oddsDrawBet',
               'odds_awin_bet365_ic': 'oddsAwinBet',
               'odds_hwin_pinn': 'oddsHwinPinn', 'odds_draw_pinn': 'oddsDrawPinn',
               'odds_awin_pinn': 'oddsAwinPinn',
               'odds_hwin_bbmax': 'oddsHwinMax', 'odds_draw_bbmax': 'oddsDrawMax',
               'odds_awin_bbmax': 'oddsAwinMax'}
    
    season_df = season_df[keepers]
    season_df.rename(columns=rename_d, inplace=True)
    
    # Calculate Results column
    conditions = [season_df['h_ftGoals'] > season_df['a_ftGoals'],
              season_df['h_ftGoals'] == season_df['a_ftGoals'],
              season_df['h_ftGoals'] < season_df['a_ftGoals']]
    choices = ['hwin', 'draw', 'awin']
    season_df['result'] = np.select(conditions, choices, default='not-played')
    
    season_df['h_result'] = season_df['result'].map({'hwin': 'win', 'draw': 'draw', 'awin': 'loss'})
    season_df['a_result'] = season_df['result'].map({'hwin': 'loss', 'draw': 'draw', 'awin': 'win'})
    
    season_df['h_goalsFor'] = season_df['h_ftGoals']
    season_df['h_goalsAgainst'] = season_df['a_ftGoals']
    
    season_df['a_goalsFor'] = season_df['a_ftGoals']
    season_df['a_goalsAgainst'] = season_df['h_ftGoals']
    
    
    # Create Implied Probability Columns and leave in odds as features
    # tot includes the bookie's overround
    tot = 1/season_df['oddsHwinMean'] + 1/season_df['oddsDrawMean'] + 1/season_df['oddsAwinMean']
    hwin = 1/season_df['oddsHwinMean']/tot
    hdraw = 1/season_df['oddsDrawMean']/tot
    adraw = 1/season_df['oddsDrawMean']/tot
    awin = 1/season_df['oddsAwinMean']/tot

    season_df['h_impliedWinProbMean'] = hwin
    season_df['h_impliedDrawProbMean'] = hdraw
    season_df['a_impliedDrawProbMean'] = adraw
    season_df['a_impliedWinProbMean'] = awin

    # Rename and split the odds columns to become features
    odds_rename = {'oddsHwinMean': 'h_oddsWinMean',
                  'oddsDrawMean': 'h_oddsDrawMean',
                  'oddsAwinMean': 'a_oddsWinMean'}

    season_df.rename(columns = odds_rename, inplace=True)
    # Add columns for away team draw
    season_df['a_oddsDrawMean'] = season_df['h_oddsDrawMean']
    new_dfs.append(season_df)
    


In [9]:
new_dfs[0].head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean
0,united-kingdom,english-premier-league,2007-2008,2007-08-11,aston-villa,liverpool,1.0,2.0,4.0,18.0,0.0,10.0,6.0,0.0,4.0,2.0,11.0,0.0,17.0,7.0,1.0,2.0,3.67,3.23,1.95,4.00,3.25,1.90,3.95,3.37,2.02,4.00,3.40,2.10,awin,loss,win,1.0,2.0,2.0,1.0,0.248863,0.282764,0.282764,0.468373,3.23
1,united-kingdom,english-premier-league,2007-2008,2007-08-11,bolton-wanderers,newcastle-united,1.0,3.0,4.0,15.0,0.0,13.0,9.0,0.0,1.0,3.0,16.0,0.0,7.0,5.0,3.0,1.0,2.41,3.18,2.80,2.50,3.20,2.75,2.60,3.10,2.90,2.60,3.25,3.05,awin,loss,win,1.0,3.0,3.0,1.0,0.381887,0.289417,0.289417,0.328696,3.18
2,united-kingdom,english-premier-league,2007-2008,2007-08-11,derby,portsmouth,2.0,2.0,6.0,14.0,0.0,12.0,5.0,1.0,1.0,6.0,17.0,0.0,12.0,6.0,1.0,2.0,2.85,3.19,2.38,2.80,3.25,2.40,3.20,3.20,2.35,3.25,3.30,2.55,draw,draw,draw,2.0,2.0,2.0,2.0,0.323531,0.289048,0.289048,0.387421,3.19
3,united-kingdom,english-premier-league,2007-2008,2007-08-11,everton,wigan-athletic,2.0,1.0,6.0,8.0,0.0,12.0,8.0,1.0,0.0,2.0,13.0,0.0,14.0,4.0,0.0,0.0,1.66,3.45,5.25,1.66,3.40,5.50,1.73,3.50,5.40,1.75,3.65,6.00,hwin,win,loss,2.0,1.0,1.0,2.0,0.556375,0.267705,0.267705,0.175920,3.45
4,united-kingdom,english-premier-league,2007-2008,2007-08-11,middlesbrough,blackburn-rovers,1.0,2.0,13.0,16.0,0.0,10.0,6.0,1.0,3.0,3.0,16.0,0.0,4.0,4.0,0.0,4.0,2.41,3.18,2.81,2.37,3.25,2.87,2.57,3.20,2.95,2.70,3.25,2.95,awin,loss,win,1.0,2.0,2.0,1.0,0.382334,0.289756,0.289756,0.327909,3.18


In [10]:
new_dfs[-1].head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean
0,united-kingdom,english-premier-league,2017-2018,2017-08-11,arsenal,leicester-city,4.0,3.0,9.0,9.0,0.0,27.0,10.0,2.0,0.0,4.0,12.0,0.0,6.0,3.0,2.0,1.0,1.51,4.43,6.44,1.53,4.5,6.50,1.38,5.19,8.92,1.55,4.60,6.89,hwin,win,loss,4.0,3.0,3.0,4.0,0.634788,0.216372,0.216372,0.148840,4.43
1,united-kingdom,english-premier-league,2017-2018,2017-08-12,brighton-and-hove-albion,manchester-city,0.0,2.0,3.0,6.0,0.0,6.0,2.0,0.0,0.0,10.0,9.0,0.0,14.0,4.0,0.0,2.0,10.10,5.25,1.32,11.00,5.5,1.33,8.09,4.98,1.42,11.50,5.60,1.36,awin,loss,win,0.0,2.0,2.0,0.0,0.094560,0.181915,0.181915,0.723525,5.25
2,united-kingdom,english-premier-league,2017-2018,2017-08-12,chelsea,burnley,2.0,3.0,8.0,16.0,2.0,19.0,6.0,0.0,3.0,5.0,11.0,0.0,10.0,5.0,3.0,3.0,1.24,6.06,13.67,1.25,6.5,15.00,1.21,7.04,15.66,1.27,6.55,15.50,awin,loss,win,2.0,3.0,3.0,2.0,0.772004,0.157968,0.157968,0.070028,6.06
3,united-kingdom,english-premier-league,2017-2018,2017-08-12,crystal-palace,huddersfield-town,0.0,3.0,12.0,7.0,0.0,14.0,4.0,0.0,1.0,9.0,19.0,0.0,8.0,6.0,2.0,3.0,1.81,3.50,4.82,1.83,3.6,5.00,1.82,3.61,4.91,1.86,3.65,5.11,awin,loss,win,0.0,3.0,3.0,0.0,0.528356,0.273236,0.273236,0.198408,3.50
4,united-kingdom,english-premier-league,2017-2018,2017-08-12,everton,stoke-city,1.0,0.0,6.0,13.0,0.0,9.0,4.0,1.0,1.0,7.0,10.0,0.0,9.0,1.0,0.0,1.0,1.69,3.69,5.50,1.70,3.8,5.75,1.74,3.96,4.94,1.71,3.85,6.00,hwin,win,loss,1.0,0.0,0.0,1.0,0.566486,0.259448,0.259448,0.174066,3.69


In [11]:
for new_df in new_dfs:
    fn = new_df['season'].unique()[0] + '.csv'
    fp = PREPPED_DATA_DIR / fn
    # print(fp)
    new_df.to_csv(fp, index=None)